In [ ]:
config_path = './config/test.json'

In [ ]:
# 依存関係のインストール
!pip -r requirements.txt

# CUDAの確認
import torch
cuda_available = torch.cuda.is_available()
print(f"CUDA is {'available' if cuda_available else 'not available'}")

if not cuda_available:
    use_zluda = input("CUDAが利用できません。WindowsでAMD GPUを使用していますか？ (y/n) ")
    if use_zluda.lower() == 'y':
        print("ZLUDAのインストールを続行します")
        %run scripts/install_zluda.py
    else:
        print("エラー：インストール中に問題が発生しました")
else:
    print("インストールが完了しました")

# 注意：この環境では%pipを使用していますが、
# 通常のコマンドラインでは!pipを使用することに注意してください

In [ ]:
# 必要なライブラリのインポート
import os

# mgdsをリポジトリにcloneした場合
# import sys
# sys.path.append('mgds/src')

import json
from modules.util.config.TrainConfig import TrainConfig
from modules.util.callbacks.TrainCallbacks import TrainCallbacks
from modules.util.commands.TrainCommands import TrainCommands
from modules.trainer.GenericTrainer import GenericTrainer

In [ ]:
# TrainConfigのインスタンスを作成
train_config = TrainConfig.default_values()
with open(config_path, "r") as f:
    train_config.from_dict(json.load(f))

# userwarningを表示しない
import warnings
warnings.filterwarnings('ignore')

# TrainConfigで読み込んだ"debug_dir","workspace_dir",cache_dir"が存在しなければ作成
for dir_path in [train_config.debug_dir, train_config.workspace_dir, train_config.cache_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# コールバックとコマンドの設定
callbacks = TrainCallbacks()
commands = TrainCommands()

# トレーニングプロセスの開始
print("Destination_path: ", train_config.output_model_destination)
print("Workspace_path: ", train_config.workspace_dir)
print("Debug_path: ", train_config.debug_dir)
print("Cache_path: ", train_config.cache_dir)

trainer = GenericTrainer(train_config, callbacks, commands)
trainer.start()

try:
    # トレーニングの実行
    trainer.train()
except Exception as e:
    print(f"トレーニング中にエラーが発生しました: {e}")
finally:
    # トレーニング終了時の処理
    trainer.end()
    print("トレーニングが終了しました")